In [1]:
%pylab inline
import numpy as np
import pandas as pd
data = []
datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.01.22.csv'
data = pd.read_csv(datapath)
data_CNF14 = data[data.Series == 'CNF14']
data = data_CNF14

Populating the interactive namespace from numpy and matplotlib


In [14]:
import time 
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()

order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
time_contain = []
time_book_sorting = []
#timestamp_.append(x[0])
y1 = data[data['TimeStamp'].str.contains(x1[0])]
y2 = data[data['TimeStamp'].str.contains(x2[0])]

bid.append(y2[(y2.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
ask.append(y1[(y1.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))

order_book_bid.append(0)
order_book_ask.append(0)
#temp_ask = 0
#temp_bid = 0
a = []
b = []
#a.append(x[0])

for i in range(len(y1),3200,1):
    #print '------------------------------------------------------------------------------'

    if data.BidOrAsk[i] == 'A':

        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                if position_ == 0:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                     
                        continue
                elif (0 < position_< (len(order_book_ask[temp_ask])-1)):
                    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                
                        continue

            elif data.TimeStamp[i] != x1[temp_ask]:
                a.append(x1[temp_ask])
                #print order_book_ask[temp_ask]
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            #print 'position and Best Price is True'
                            continue
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1

                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                        
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                        
                            continue
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                #print temp_ask
                #print order_book_ask[temp_ask]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]

                if position_ == 0:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                        
                        continue

            elif data.TimeStamp[i] != x1[temp_ask]:
                a.append(x1[temp_ask])
                #print temp_ask
                #print order_book_ask[temp_ask]
                order_book_ask.append(order_book_ask[temp_ask])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                #print 'temp_ask2 = %d'%(temp_ask)
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]
                
                if position_ == 0:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                        
                        continue                

    elif data.BidOrAsk[i] == 'B':
        #print i
        #print int(data[['QuantityDifference']][i:i+1].values) > 0
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            #print data.TimeStamp[i] == x2[temp_bid]
            
            if data.TimeStamp[i] == x2[temp_bid]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                #print position_
                
                if position_ == 0:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error2 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error3 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                
                        continue

            elif data.TimeStamp[i] != x2[temp_bid]:
                b.append(x2[temp_bid])
                #print 'temp bid = %d'%(temp_bid)
                #print order_book_bid[temp_bid]
                if temp_bid == 0:
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_bid[temp_bid]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print position_
                    #print len(order_book_bid[temp_bid]) - 1
                    
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    if position_ == 0:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error1 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                            
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            #print 'position and Best Price is True'
                            continue
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_    
                    #print len(order_book_bid[temp_bid])-1      
                    if position_ == 0:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                        
                            continue
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                #print temp_bid
                #print order_book_bid[temp_bid]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]

                if position_ == 0:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                        
                        continue

            elif data.TimeStamp[i] != x2[temp_bid]:
                b.append(x2[temp_bid])
                #print temp_bid
                #print order_book_bid[temp_bid]
                order_book_bid.append(order_book_bid[temp_bid])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print 'temp_bid2 = %d'%(temp_bid)
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                if position_ == 0:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                     
                        continue
    
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        #print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        #print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        #print 'position and Best Price is True'                        
                        continue                
                                    
end = time.time()            
print "Total time = %f"%(end - start)    

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:346: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:389: SettingWithCop

Total time = 28.867115


In [15]:
len(order_book_ask)

1314

In [16]:
len(order_book_bid)

969

In [17]:
len(b)

968

In [18]:
len(a)

1313